В даному ноутбуку вказано учбову реалізацію запропонованої в випускній-кваліфікаційній роботі
системи аналізу настроїв у текстових повідомленнях
з використанням 2 напрямків: загальної класифікації типу
"позитивне"/"негативне" чи спеціалізованої емоції типу
"злість", "сум".

Система підтримує моделі для аналізу настроїв у текстових повідомленнях, тренованих на українських чи англомовних наборах даних, які були відповідно розмічені перед тренуванням моделей.

Для правильного функціонування рекомендується читати підказки в текстових клітинках та виконувати дії послідовно.
Тренування власних моделей для аналізу настроїв у текстових повідомленнях винесено в окремі ноутбуки:
- english training: https://colab.research.google.com/drive/1xLJEz-VzsutVU9KdHLAOIsrQ7WBISftw?usp=sharing
- ukranian training: https://colab.research.google.com/drive/1aw7bWfR4n12vzYl2z1_VGZKJIFhECvZK?usp=sharing

# Train your english emotion analysis model here.

STEP #1: Run next cell to setup needed libraries for work.

In [ ]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from tensorflow.keras.models import Sequential


STEP #2: Load your data: train, validate, test files.

In [ ]:
#train_texts, train_labels = load_file('train.txt')
#val_texts, val_labels = load_file('val.txt')
#test_texts, test_labels = load_file('test.txt')

NameError: name 'load_file' is not defined

STEP #3: Use next cell to prepare data for proper usage for build-up model

In [ ]:
train_texts, train_labels = load_file('train.txt')
val_texts, val_labels = load_file('val.txt')
test_texts, test_labels = load_file('test.txt')

In [ ]:
# Convert labels to numerical values
label_to_index = {'sadness': 0, 'joy': 1, 'anger': 2, 'fear': 3}
train_labels = [label_to_index[label] for label in train_labels]
val_labels = [label_to_index[label] for label in val_labels]
test_labels = [label_to_index[label] for label in test_labels]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad the sequences
max_length = max([len(seq) for seq in train_sequences + val_sequences + test_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

STEP #4: Training your model: may take some time

In [ ]:
# Cell 4: Build and train the RNN (LSTM) model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))  # 4 classes: sadness, joy, anger, fear

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_padded, np.array(train_labels), epochs=10, batch_size=32, validation_data=(val_padded, np.array(val_labels)))

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, np.array(test_labels))
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

STEP #5: Test your model on real examples:

In [ ]:
# Cell 5: Test the model with user input
user_input = input("Enter a text to test the model: ")
user_sequence = tokenizer.texts_to_sequences([user_input])
user_padded = pad_sequences(user_sequence, maxlen=max_length, padding='post')
prediction = model.predict(user_padded)
predicted_label = np.argmax(prediction)
label_index = {0: 'sadness', 1: 'joy', 2: 'anger', 3: 'fear'}
print(f"Predicted emotion: {label_index[predicted_label]}")

STEP #6 (opt) Save your model and download it to use on colab/local system in .h5 format.

In [ ]:
import pickle
with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

model.save('nlp.h5')